#Local Field Potential

Installing the allen institute SDK for reaching the visual coding data.

**Notice!** you must restart the kernel after finishing the allensdk installation, due to the change of the pandas version

In [ ]:
!pip install allensdk==2.9.0

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from allensdk.brain_observatory.ecephys.ecephys_session import EcephysSession
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_project_api.utilities import build_and_execute
from allensdk.brain_observatory.ecephys.ecephys_project_api.rma_engine import RmaEngine
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot

Mount your google drive, so you can save the cache files there and re-use them later. Make sure you have ~3GB free space on your chosen google drive account.

In [ ]:
data_directory = '/content/drive/MyDrive/ecephys_cache/' # must be updated to a valid directory in your filesystem
manifest_path = os.path.join(data_directory, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [ ]:
session_id = 750749662
session = cache.get_session_data(session_id)

Print the probes of the session along with the areas they recorded from

In [ ]:
{session.probes.loc[probe_id].description : 
     list(session.channels[session.channels.probe_id == probe_id].ecephys_structure_acronym.unique())
     for probe_id in session.probes.index.values}

##LFP
The local field potential is the extracellular current flow that reflects the linearly summed post-synaptic currents from local cell groups.

In [ ]:
probe_id = session.probes.query('description=="probeF"').index.values[0]
lfp_sampling_frequency = session.probes.loc[probe_id, 'lfp_sampling_rate']
lfp = session.get_lfp(probe_id)

###Aligning LFP data to a stimulus


We'll use the flashes stimulus for showing the changes of LFP by stimulus

In [ ]:
presentation_table = session.stimulus_presentations.query("stimulus_name == 'flashes'")
presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values

In [ ]:
trial_window = np.arange(-0.5, 0.5, 1/500)
time_selection = np.concatenate([trial_window + t for t in presentation_times])

inds = pd.MultiIndex.from_product((presentation_ids, trial_window), 
                                  names=('presentation_id', 'time_from_presentation_onset'))

ds = lfp.sel(time = time_selection, method='nearest').to_dataset(name = 'aligned_lfp')
ds = ds.assign(time=inds).unstack('time')

aligned_lfp = ds['aligned_lfp']

In [ ]:
plt.figure(figsize=(8,6))
n_channels = len(aligned_lfp.channel)
im = plt.imshow(aligned_lfp.mean(dim='presentation_id'), aspect='auto', 
                origin='lower', vmin=-1e-4, vmax=1e-4, 
                extent=[trial_window[0], trial_window[-1], 0, n_channels])
plt.colorbar(im, fraction=0.036, pad=0.04)
channels_idx = np.arange(0, n_channels, 5)
plt.yticks(channels_idx, aligned_lfp.channel[channels_idx].values)
plt.xlabel('Time after stimulus onset [s]')
plt.ylabel('Channel index')
plt.show()

We can see in the above plot that there are some units that fire constantly, regardless of the stimulus presented, let's check their type:

In [ ]:
session.channels.loc[850165396]

In [ ]:
aligned_lfp

##Brain Oscillations
**Delta**: 0.5 - 4Hz - Usually associated with the deep stage 3 of NREM sleep (SWS) or quiet wakefulness. Delta waves are associated with the cortex, however can sometimes be measured in sub-cortical areas like hippocampus, etc.  
**Theta**: 4 - 8Hz - Observed in the hippocampus but can also be detected in numerous other cortical and subcortical brain structures. Hippocampal theta waves, with a frequency range of 6–10 Hz, appear when a rat is engaged in active motor behavior such as walking or exploratory sniffing, and also during REM sleep. Theta waves with a lower frequency range, usually around 6–7 Hz, are sometimes observed when a rat is motionless but alert.  
**Alpha**: 8 - 12 Hz - arch-shaped alpha frequency oscillations are observed in rat somatosensory cortex, and these have been hypothesized to be analogous to the mu rhythms in EEG.  
**Beta**: 15 - 30Hz - Beta states are the states associated with normal waking consciousness. Over the motor cortex beta waves are associated with the muscle contractions that happen in isotonic movements and are suppressed prior to and during movement changes.  
**Low Gamma**: 25 - 50 Hz  
**High Gamma**: 65 - 140 Hz


##Pupil Data and Brain States
Pupil diameter can be a good estimator for arousel.  
Arousal measures accurately predicted multiple modes of membrane potential activity, including rhythmic slow oscillations at low arousal, stable hyperpolarization at intermediate arousal, and depolarization during phasic or tonic periods of hyper-arousal.  
   
McGinley, M. J., David, S. V., & McCormick, D. A. (2015). Cortical Membrane Potential Signature of Optimal States for Sensory Signal Detection. Neuron, 87(1), 179–192. https://doi.org/10.1016/j.neuron.2015.05.038

In [ ]:
pupil_data = session.get_pupil_data()
pupil_data.index.rename('time', inplace=True)
pupil_data.head()

###Pupil Simulation

In [ ]:
%%capture
from matplotlib import animation
from matplotlib.patches import Ellipse

def plot_animated_ellipse_fits(pupil_data: pd.DataFrame, start_frame: int, end_frame: int):

    start_frame = 0 if (start_frame < 0) else start_frame
    end_frame = len(pupil_data) if (end_frame > len(pupil_data)) else end_frame
    
    frame_times = pupil_data.index.values[start_frame:end_frame]
    interval = np.average(np.diff(frame_times)) * 1000

    fig = plt.figure()
    ax = plt.axes(xlim=(0, 480), ylim=(0, 480))

    cr_ellipse = Ellipse((0, 0), width=0.0, height=0.0, angle=0, color='white')
    pupil_ellipse = Ellipse((0, 0), width=0.0, height=0.0, angle=0, color='black')
    eye_ellipse = Ellipse((0, 0), width=0.0, height=0.0, angle=0, color='grey')

    ax.add_patch(eye_ellipse)
    ax.add_patch(pupil_ellipse)
    ax.add_patch(cr_ellipse)

    def update_ellipse(ellipse_patch, ellipse_frame_vals: pd.DataFrame, prefix: str):
        ellipse_patch.center = tuple(ellipse_frame_vals[[f"{prefix}_center_x", f"{prefix}_center_y"]].values)
        ellipse_patch.width = ellipse_frame_vals[f"{prefix}_width"]
        ellipse_patch.height = ellipse_frame_vals[f"{prefix}_height"]
        ellipse_patch.angle = np.degrees(ellipse_frame_vals[f"{prefix}_phi"])
    
    def init():
        return [cr_ellipse, pupil_ellipse, eye_ellipse]

    def animate(i):
        ellipse_frame_vals = pupil_data.iloc[i]
        
        update_ellipse(cr_ellipse, ellipse_frame_vals, prefix="corneal_reflection")
        update_ellipse(pupil_ellipse, ellipse_frame_vals, prefix="pupil")
        update_ellipse(eye_ellipse, ellipse_frame_vals, prefix="eye")
        
        return [cr_ellipse, pupil_ellipse, eye_ellipse]
    
    return animation.FuncAnimation(fig, animate, init_func=init, interval=interval, frames=range(start_frame, end_frame), blit=True)

anim = plot_animated_ellipse_fits(pupil_data, 100, 600)

In [ ]:
from IPython.display import HTML

HTML(anim.to_jshtml())

## Gaze Data

In [ ]:
gaze_data = session.get_screen_gaze_data()
gaze_data.index.rename('time', inplace=True)
gaze_data

Plotting the gaze vector on top of the shown natural scene, shows the saccades of the mouse.

In [ ]:
natural_presentations = session.stimulus_presentations.query('stimulus_name=="natural_scenes"')
pid = 3
owl_frame = 58.0
image = cache.get_natural_scene_template(owl_frame)
plt.figure(figsize=(10, 10))
plt.imshow(image, cmap=plt.cm.gray, extent=[20, -30, -20, 10])

for pid in natural_presentations.query(f'frame=={owl_frame}').index:
  start_time = natural_presentations.start_time.loc[pid]
  end_time = natural_presentations.stop_time.loc[pid]
  gaze_data1 = gaze_data.query(f'time>={start_time} and time<={stop_time}')
  plt.plot(gaze_data1.raw_screen_coordinates_x_cm, gaze_data1.raw_screen_coordinates_y_cm)

### Signal Processing Function

In [ ]:
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def window_rms(a, window_size):
  a2 = np.power(a, 2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'same'))

For the following calculation we'll use a segment of time start from the last presentation of the owl and ended 50 seconds after.

In [ ]:
end_time = start_time + 50

Extracting the LFP data from one of the channels in the vicinity of CA1

In [ ]:
lfp_data = lfp.sel(time = slice(start_time, end_time))
channel_id = 850165396  # CA1
lfp_data = lfp_data.sel(channel=channel_id, method='nearest')
lfp_sampling_frequency = session.probes.loc[probe_id, 'lfp_sampling_rate']

Same for the pupil width

In [ ]:
pupil_width = pupil_data.query(f'time>={start_time} and time<={end_time}').pupil_width

Getting the Delta and Theta waves for the above metioned time segment and plotting them alongside the pupil diameter.

In [ ]:
waves = {
  'delta': butter_bandpass_filter(lfp_data, 0.5, 4, lfp_sampling_frequency, order=3),
  'theta': butter_bandpass_filter(lfp_data, 6, 10, lfp_sampling_frequency, order=3)
}

fig, axes = plt.subplots(3,1,figsize=(20, 3*3))
axes[0].plot(pupil_width.index, pupil_width.values)
axes[0].set_title('Pupil Diameter')
for i, (wave_name, sig) in enumerate(waves.items()):
  axes[i+1].plot(lfp_data.time, sig)
  axes[i+1].set_title(wave_name)

fig.tight_layout()

Calculating the correlation of the pupil diameter with brain waves RMS

In [ ]:
window_size = int(0.5 * lfp_sampling_frequency)

for wave_name, sig in waves.items():
  x_rms = window_rms(sig, window_size)
  v = np.interp(pupil_width.index, lfp_data.time.values, x_rms)
  x = np.vstack([pupil_width.values, v])
  cor = np.corrcoef(x)
  print(f'Correlation for {wave_name}: {cor[0,1]:.2f}')